In [ ]:
!uv add matplotlib --active

In [ ]:
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
import scipy.stats as stats

In [ ]:
targets = ["postgres", "databricks"]
experiments = ["projection", "selection", "cross_join", "inner_join", "deduplication"]

experiment_results = {}

for target in targets:
    experiment_results[target] = {}
    for experiment in experiments:
        with open(f"../experiment_results/{target}/{experiment}/execution_times.pkl", "rb") as file:
            execution_times = pickle.load(file)
        experiment_results[target][experiment] = execution_times

experiment_results

In [ ]:
labels = {
    "projection": "Projection",
    "selection": "Selection",
    "cross_join": "Cartesian Product",
    "inner_join": "Join",
    "deduplication": "Deduplication",
}

def get_label(dataset):
    return labels[dataset]

## Individual Tables

In [ ]:
for target in targets:
    rows = []
    for experiment in experiments:
        results = experiment_results[target][experiment]
        for dataset, times in results.items():
            average_time = np.mean(times)
            low, high = stats.t.interval(
                0.95,
                df=len(times)-1,
                loc=average_time,
                scale=np.std(times, ddof=1) / np.sqrt(len(times))
            )
            row = {
                "Experiment": experiment,
                "Dataset": dataset,
                "Average Time": average_time,
                "95% Conf. Int. Low": low,
                "95% Conf. Int. High": high,
            }
            rows.append(row)
    df = pd.DataFrame(rows)
    print(target)
    display(df)
    print(df.to_latex(
        index=False,
        formatters={ "Experiment": get_label },
        float_format="{:.3f}".format
    ))

## Combined Table

In [ ]:
rows = []
for experiment in experiments:
    results_postgres = experiment_results["postgres"][experiment]
    results_databricks = experiment_results["databricks"][experiment]
    for dataset in results_postgres.keys():
        times_postgres = results_postgres[dataset]
        times_databricks = results_databricks[dataset]
        average_time_postgres = np.mean(times_postgres)
        low_postgres, high_postgres = stats.t.interval(
            0.95,
            df=len(times_postgres)-1,
            loc=average_time_postgres,
            scale=np.std(times_postgres, ddof=1) / np.sqrt(len(times_postgres))
        )
        
        times_databricks = results_databricks[dataset]
        times_databricks = results_databricks[dataset]
        average_time_databricks = np.mean(times_databricks)
        low_databricks, high_databricks = stats.t.interval(
            0.95,
            df=len(times_databricks)-1,
            loc=average_time_databricks,
            scale=np.std(times_databricks, ddof=1) / np.sqrt(len(times_databricks))
        )

        t_stat, p_value = stats.ttest_ind(times_databricks, times_postgres)
        
        row = {
            "Experiment": experiment,
            "Dataset": dataset,
            "Doubtless Average Time": average_time_databricks,
            "DuBio Average Time": average_time_postgres,
            "P-Value": p_value
        }
        rows.append(row)
df = pd.DataFrame(rows)
print("combined")
display(df)
print(df.to_latex(
    index=False,
    formatters={ "Experiment": get_label },
    float_format="{:.3f}".format
))